In [1]:
from brainlit.utils import upload, session
from pathlib import Path

## Uploading Brain Images from data in the Octree format.
This notebook demonstrates uploading the 2 lowest-resolution brain volumes, as well as a `.swc` segment file.
The upload destination could easily be set to a url of a cloud data server such as s3.

## 1) Define variables.
 - `source` and `source_segments` are the root directories of the octree-formatted data and swc files.
 - `p` is the prefix string. `file://` indicates a filepath, while `s3://` or `gc://` indicate URLs.
 - `dest` and `dest_segments` are the destinations for the uploads (in this case, filepaths).
 - `num_res` denotes the number of resolutions to upload.

In [2]:
source = (Path().resolve().parents[2] / "data" / "data_octree").as_posix()
dest = (Path().resolve().parents[2] / "data" / "upload").as_uri()
dest_segments = (Path().resolve().parents[2] / "data" / "upload_segments").as_uri()
dest_annotation = (Path().resolve().parents[2] / "data" / "upload_annotation").as_uri()
num_res = 2

## 2) Upload the image data.

In [3]:
upload.upload_volumes(source, dest, num_res)

Creating precomputed volume:   0%|          | 0/8 [00:00<?, ?it/s]
Finished mip 0, took 5.183950901031494 seconds
Creating precomputed volume:  62%|██████▎   | 5/8 [00:12<00:07,  2.38s/it]

In [ ]:
upload.upload_segments(source, dest_segments, num_res)

In [ ]:
_, _, voxel_size, img_size, _ = upload.get_volume_info(source, num_res)
upload.create_cloud_volume(dest_annotation, img_size, voxel_size, 1, layer_type="annotation")

In [17]:
%%capture
from brainlit.utils.session import NeuroglancerSession
from brainlit.algorithms.generate_fragments.tube_seg import tubes_seg
sess = NeuroglancerSession(url = dest, url_segments= dest_segments, url_annotation=dest_annotation, mip=0)
img, bounds, vertices = sess.pull_vertex_list(2, range(100, 200), expand=True)  # get image containing some data
labels = tubes_seg(img, vertices, radius=.1)  # generate labels via tube segmentation
sess.push(labels, bounds)  # push labels

## Visualize your data with NeuroglancerSession and napari

In [18]:
from cloudvolume import Bbox
from brainlit.utils.swc import graph_to_paths
import napari

In [19]:
G_sub = sess.get_segments(2, bounds)
paths = graph_to_paths(G_sub)

Downloading: 100%|██████████| 1/1 [00:00<00:00, 518.46it/s]


In [16]:
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_labels(labels)
    viewer.add_shapes(data=paths, shape_type='path', edge_width=0.1, edge_color='blue', opacity=0.1)